This notebook was developed by Gautam Machiraju and Conor Corbin, modified by Minh Nguyen

### Description: major changes
Because binning relies on training data, we need to *use all the old and new data* here
- Use 2015 - *2018* as the first training data to check for value distributions with 10 quantiles 
- Use these distributions to assign new validation and test values to quantile bins 
- 2 major steps:
    - With the old data, use 2015 - 2018 as the first training data to create quantile distribution, and 2019 - 03/2020 as validation data. This is for modeling training to tune hyperparameters
    - Comebine the new data with old data, use 2015 - 03/2020 as train/val data to create quantile distribution, and 04/2020 - 2021 as the test data, assign their binns from the the train/val data.
    
**Input:** 
- `6_9_coh4_feature_values`: used new cohort4
- `2_7_coh4_feature_values`: used old cohort4

**Outputs:** `...coh4...`
- `6_10_binned_labs_vitals_train`:
    - use 2015 - *2018* as trained data for value distributions to bin validation data of 2019-03/2020
    - this is for the purpose of train data and select model hyper parameters on validation data
    - validation data > 2018 (2019 - 03/2020) 
- `6_10_binned_labs_vitals_test`:
    - after tuning hyperparameters, the value distributions are done again on data trai/val data from 2015 - 03/2020
    - test data (04/2020 - 2021) are binned on based on this distribution
    - binned test data is used for the final prediction and evaluation of the new model after THICK DESCRIPTION work
- Note that the size of these 2 data sets are different

In [1]:
import os 
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
import os 
from google.cloud import bigquery
from google.cloud.bigquery import dbapi

##Use correct path based on whether you are, Nero or local
# use Ctrl + Insert to copy and Shift + Insert to paste

# for Nero:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/minh084/.config/gcloud/application_default_credentials.json' 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jupyter/.config/gcloud/application_default_credentials.json'

# for local computer:
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\User\AppData\Roaming\gcloud\application_default_credentials.json' 

##set correct Nero project
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-jonc101' 

In [4]:
outdir = "../../OutputTD/shc2021"

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

### Train And Apply Featurizer - change for new validation data after March 2020! 

#### Depending on which dataset used for train/val or train_val/test, choose different time split
- For the purpose of training to select hyperparameters for models, use 2015 - 2017 (<2018) for binning distribution --> **NEW: use 2015 - 2018**
- One a final model with hyperparameters are selected, for the final prediction on an unseen test set, use 2015 - 2018 for binning distribution ( < 2019) --> **NEW: use 2015 - 03/2020**
- basically with this new validation data, we could use all the old data in this process, without saving the 2019 - 03/2020 as the final test set

In [84]:
df = pd.read_csv(os.path.join(outdir, "12_coh4_noOR_noDKA_feature_values.csv"))

# need this if read from local file, if pulled from BQ, already in datetime format
df["admit_time"] = pd.to_datetime(df["admit_time"], utc=True) 
df["time"] = pd.to_datetime(df["time"])

print(len(df)) # 3867101
print(df.pat_enc_csn_id_coded.nunique())

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (2,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


3867101
51846


In [16]:
df.dtypes

anon_id                                 object
pat_enc_csn_id_coded                     int64
admit_time                 datetime64[ns, UTC]
death_3hr_recent_label                 float64
death_6hr_recent_label                 float64
death_12hr_recent_label                float64
death_24hr_recent_label                float64
feature_type                            object
features                                object
values                                 float64
time                       datetime64[ns, UTC]
dtype: object

In [85]:
# check to see if any feature is null
df[df['features'].isnull()]

,anon_id,pat_enc_csn_id_coded,admit_time,death_3hr_recent_label,death_6hr_recent_label,death_12hr_recent_label,death_24hr_recent_label,feature_type,features,values,time


In [6]:
# df.loc[df['pat_enc_csn_id_coded'] == 131250899044]

In [86]:
df = df[df['feature_type'].isin(['labs', 'vitals'])]
print(len(df)) #2778335
print(df.pat_enc_csn_id_coded.nunique())

2778335
51846


In [87]:
df.head(3)

,anon_id,pat_enc_csn_id_coded,admit_time,death_3hr_recent_label,death_6hr_recent_label,death_12hr_recent_label,death_24hr_recent_label,feature_type,features,values,time
1088766,JC1000116,131066472308,2015-01-28 00:46:00+00:00,0.0,0.0,0.0,0.0,vitals,DBP,57.0,2015-01-27 07:30:00+00:00
1088767,JC1000116,131066472308,2015-01-28 00:46:00+00:00,0.0,0.0,0.0,0.0,vitals,SBP,105.0,2015-01-27 07:30:00+00:00
1088768,JC1000116,131066472308,2015-01-28 00:46:00+00:00,0.0,0.0,0.0,0.0,vitals,DBP,65.0,2015-01-27 07:00:00+00:00


In [89]:
# calculate time difference in hours between admit time and time of vitals and labs,
df['difftime'] = round((df['admit_time'] - df['time']).dt.total_seconds()/3600, 1)

# check for observations more than 24 hours before admission time
print(len(df.loc[df['difftime'] > 24])) # 40070
print(df.pat_enc_csn_id_coded.nunique())
df[['difftime']].describe()

46216
51846


,difftime
count,2.778335e+06
mean,4.100405e+00
std,8.435068e+01
min,0.000000e+00
25%,1.300000e+00
50%,2.500000e+00
75%,4.400000e+00
max,1.401405e+05


In [90]:
# remove those observations:
df = df.loc[df['difftime'] <= 24]
print(len(df)) #2732119
print(df.pat_enc_csn_id_coded.nunique()) # there is one patient who did not have labs or vitals within 24 hours?

# check for coh4:
2778335 - 46216

2732119
51845


2732119

### Process Continuous Features
Create a function that "trains" binning featurizer (computes distribution of values) based on subset of the data.  This is important because we only want to build the distribution with our training set and apply the bin featurizer
to the test set (prevents leakage). 

Then create a function that "applies" the trained featurizer on a set of data. 


In [22]:
def convert_to_dict(look_up_table):
    """Converts df look up table to dictionary for faster look up later"""
    bin_val_dict = {}
    for feature in look_up_table['features'].unique():
        feature_bin_vals = look_up_table[look_up_table['features'] == feature]
        for _bin in feature_bin_vals['bins'].unique():
            if feature not in bin_val_dict:
                bin_val_dict[feature] = {}
                bin_val_dict[feature]['min'] = []
                bin_val_dict[feature]['max'] = []

            min_val_for_bin = feature_bin_vals[feature_bin_vals['bins'] == _bin]['values']['min'].values[0]
            max_val_for_bin = feature_bin_vals[feature_bin_vals['bins'] == _bin]['values']['max'].values[0]

            bin_val_dict[feature]['min'].append(min_val_for_bin)
            bin_val_dict[feature]['max'].append(max_val_for_bin)
    return bin_val_dict

    
def train_featurizer(df_train):
    """
    Compute percent_ranks and generates a look up table of min and max bin values
    Input : long form dataframe with features and values where values are the continuous values of labs / vitals
    Output: look up table - dict of dict of lists (key1 = feature_name, key2 = max or min, values = lists of values)
    """
    # Compute percentiles and bins
    df_train['percentiles'] = df_train.groupby('features')['values'].transform(lambda x: x.rank(pct=True))
    df_train['bins'] = df_train['percentiles'].apply(lambda x: int(x * 10))
    
    # Generate look up table and conver to dictionary stucture
    look_up_table_df = df_train.groupby(['features', 'bins']).agg({'values' : ['min', 'max']}).reset_index()
    look_up_table = convert_to_dict(look_up_table_df)
    
    ### Sanity Check. Ensure that min vector for each feature is strictly increasing (no ties!)
    # Should be the case because ties are given same percentile rank in default pandas rank function
    for feature in look_up_table:
        mins = look_up_table[feature]['min']
        for i in range(len(mins)-1):
            assert mins[i] < mins[i+1]
    
    return look_up_table


def apply_featurizer(df, look_up_table):
    
    def get_appropriate_bin(feature, value, look_up_table):
        """Takes in feature, value and look up table and returns appropriate bin

        Quick Note: For some features, we do not have 10 bins.  This happens when we have many many ties in the 
        percent rank - and the percent rank alg returns ties as the average rank within that tie. So for instance
        we're trying to break each feature up into deciles where each bin covers range of 10% of the examples. But if more
        than 10% of the examples take on 1 value - then bins can be skipped. This shouldn't really be a problem
        for downstream tasks - just something to be aware of. This also means 'bins' and 'bins_applied' won't have
        perfect overlap in features that end up having less than 10 bins

        """
        mins = look_up_table[feature]['min']
        for i in range(len(mins) - 1):
            # If value is smaller than min value of smallest bin (in test time) - then return 0 (smallest bin)
            if i == 0 and value < mins[i]:
                return i

            if value >= mins[i] and value < mins[i+1] :
                return i

        # Then in last bin
        return len(mins)-1
    
    df['bins_applied'] = df[['features', 'values']].apply(
        lambda x: get_appropriate_bin(x['features'], x['values'], look_up_table), axis=1)
    
    return df

In [68]:
# sanity check of the total cohort size with time splitting
# Change the years
print(len(df))
print(len(df[(df['admit_time'].dt.year < 2020)]))
print(len(df[(df['admit_time'].dt.year < 2021)]))

print(len(df[(df['admit_time'].dt.year == 2020)]))
print(len(df[(df['admit_time'].dt.year > 2020)]))
len(df[(df['admit_time'].dt.year < 2020)]) + len(df[(df['admit_time'].dt.year == 2020)]) + len(df[(df['admit_time'].dt.year > 2020)])

2732119
1866392
2319725
453333
412394


2732119

### Train And Apply Featurizer - change for new validation data after March 2020! 

#### Depending on which dataset used for train/val or train_val/test, choose different time split
- For the purpose of training to select hyperparameters for models, use 2015 - 2019 for binning distribution
- One a final model with hyperparameters are selected, for the final prediction on an unseen test set, use 2015 - 2020 for binning distribution

In [12]:
# for train/val VS. train_val/test:
# df_train =  df[(df['admit_time'].dt.year < 2019)] # old 2018
# df_train =  df[(df['admit_time'].dt.year < 2021)] # old 2019

In [52]:
# FOR TRAINING/VALIDATION purpose - chosing hyperparameters, 
# do decile distribution only on train set
df_train =  df[(df['admit_time'].dt.year < 2020)]

In [70]:
# continue here
look_up_table = train_featurizer(df_train)
df_featurized = apply_featurizer(df, look_up_table)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Quick Sanity Check
For features that have 10 bins from 0 to 9 - `bins` should be same as `bins_applied`

In [71]:
df_train = apply_featurizer(df_train, look_up_table)
look_up_table_df = df_train.groupby(['features', 'bins']).agg({'values' : ['min', 'max']}).reset_index()

features_with_0_9_bins = []
for feature in look_up_table_df:
    num_bins = len(look_up_table_df[look_up_table_df['features'] == feature]['bins'].values)
    ten_in_bins = 10 in look_up_table_df[look_up_table_df['features'] == feature]['bins'].values
    if num_bins == 10 and not ten_in_bins:
        features_with_0_9_bins.append(feature)

for feature in features_with_0_9_bins:
    df_test = df_train[df_train['features'] == 'feature']
    for b_real, b_computed in zip(df_test['bins'].values, df_test['bins_applied'].values):
        assert(b_real == b_computed)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [72]:
# checked against R 2.9_
len(df_train) # train 1866392 vs train+val 2319725

2319725

### Little bit of house cleaning
Create new feature names that reflect which bin the value belongs in

In [73]:
columns = ['anon_id', 'pat_enc_csn_id_coded', 'admit_time', 'feature_type', 'features', 'values', 'bins_applied']
df_new = df_featurized[columns]

In [74]:
df_new['features'] = ['_'.join([x, str(y)]) for x, y in zip(df_new['features'].values, df_new['bins_applied'].values)] 

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [75]:
len(df_new) #2732119 same

2732119

### Get Counts representation
Group by patient, cns, and feature name (with bin value appended to feature name) and make value the number of times
that particular feature appears for that csn id. 

In [76]:
df_final = df_new.groupby(['anon_id', 'pat_enc_csn_id_coded', 'features']).agg(
    {'admit_time' : 'first',
     'feature_type' : 'first',
     'values' : 'count'}).reset_index()

columns = ['anon_id', 'pat_enc_csn_id_coded', 'admit_time', 'feature_type', 'features', 'values']
df_final = df_final[columns] # reorder columns
 
df_final['feature_type'] = [x + '_results' if x == 'labs' else x for x in df_final['feature_type'].values]

#### Depending on which dataset used for train/val or train_val/test, name: _train vs _test
- Rename feature_type to reflect training set used: ..._train or ...test
- 'vitals_train' means everything up to 2019 used. (train vs val)
- 'vitals_test' means everything up to 2020 used. (train + val vs test)

In [60]:
# for train/val to select hyperparameters, binned distributions on train/val: 
# df_train =  df[(df['admit_time'].dt.year < 2018)] - old original
df_final['feature_type'] = [x + '_train' for x in df_final['feature_type'].values]

In [78]:
print(len(df_final)) # 2,185,308 vs 2,184,675 why less than train???
print(len(df)) #2732119 same

2184675
2732119


In [79]:
# Sanity check - sum of the counts should be length of the orginal dataframe
assert df_final['values'].sum() == len(df)

In [80]:
df_final.head(10)

,anon_id,pat_enc_csn_id_coded,admit_time,feature_type,features,values
0,JC1000116,131066472308,2015-01-28 00:46:00+00:00,labs_results_test,ALB_1,1
1,JC1000116,131066472308,2015-01-28 00:46:00+00:00,labs_results_test,ALB_2,1
2,JC1000116,131066472308,2015-01-28 00:46:00+00:00,labs_results_test,ALB_4,1
3,JC1000116,131066472308,2015-01-28 00:46:00+00:00,labs_results_test,ALK_5,1
4,JC1000116,131066472308,2015-01-28 00:46:00+00:00,labs_results_test,ALK_6,2
5,JC1000116,131066472308,2015-01-28 00:46:00+00:00,labs_results_test,ALT_2,1
6,JC1000116,131066472308,2015-01-28 00:46:00+00:00,labs_results_test,ALT_4,2
7,JC1000116,131066472308,2015-01-28 00:46:00+00:00,labs_results_test,AST_0,1
8,JC1000116,131066472308,2015-01-28 00:46:00+00:00,labs_results_test,AST_1,2
9,JC1000116,131066472308,2015-01-28 00:46:00+00:00,labs_results_test,AnionGap_0,2


### SAVE, depending on which cohort and which dataset

#### cohort4, train/val, train_val/test

In [64]:
# for training to select hyperparameters: df_train =  df[(df['admit_time'].dt.year < 2019)] - new
# no OR no DKA cohort 4 final
df_final.to_csv(os.path.join(outdir, '13_coh4_binned_labs_vitals_train.csv'), index=False)

### ReDO with TRAIN_VAL/TEST

- First, format the new data (04/2020 - 2021)
- Remove the bins_applied column from the old data
- Concat the 2 data into one for re-binning

In [67]:
### OR 2nd time on TRAIN_VAL/TEST
# AFTER training and chosing hyperparameters, RE-DO DECILE distribution on both train_val set (2015 - 2018)
# df_train =  df[(df['admit_time'].dt.year < XXX)] # all old data up to 03/2020
df_train =  df[(df['admit_time'].dt.year < 2021)]
print(len(df_train)) # 2319725

# continue here
look_up_table = train_featurizer(df_train)
df_featurized = apply_featurizer(df, look_up_table)  # NOTE, df here is the combined df0 and df21

# sanity check
df_train = apply_featurizer(df_train, look_up_table)
look_up_table_df = df_train.groupby(['features', 'bins']).agg({'values' : ['min', 'max']}).reset_index()

features_with_0_9_bins = []
for feature in look_up_table_df:
    num_bins = len(look_up_table_df[look_up_table_df['features'] == feature]['bins'].values)
    ten_in_bins = 10 in look_up_table_df[look_up_table_df['features'] == feature]['bins'].values
    if num_bins == 10 and not ten_in_bins:
        features_with_0_9_bins.append(feature)

for feature in features_with_0_9_bins:
    df_test = df_train[df_train['features'] == 'feature']
    for b_real, b_computed in zip(df_test['bins'].values, df_test['bins_applied'].values):
        assert(b_real == b_computed)

# cleaning - 
# create new feature names that reflect which bin the value belongs in
columns = ['anon_id', 'pat_enc_csn_id_coded', 'admit_time', 'feature_type', 'features', 'values', 'bins_applied']
df_new = df_featurized[columns]

df_new['features'] = ['_'.join([x, str(y)]) for x, y in zip(df_new['features'].values, df_new['bins_applied'].values)] 

print(len(df_new)) # 2732119

# get count presentation
df_final = df_new.groupby(['anon_id', 'pat_enc_csn_id_coded', 'features']).agg(
    {'admit_time' : 'first',
     'feature_type' : 'first',
     'values' : 'count'}).reset_index()

columns = ['anon_id', 'pat_enc_csn_id_coded', 'admit_time', 'feature_type', 'features', 'values']
df_final = df_final[columns] # reorder columns

df_final['feature_type'] = [x + '_results' if x == 'labs' else x for x in df_final['feature_type'].values]

# # OR 2nd time on train-val/test: rename feature type to reflect train or test
df_final['feature_type'] = [x + '_test' for x in df_final['feature_type'].values]

print(len(df_final)) # 2184675 (2534674 old 6_validation total)
print(len(df)) # 2732119

# sanity check
assert df_final['values'].sum() == len(df)

### OR 2nd time on train-val/test
df_final.to_csv(os.path.join(valdir, '13_coh4_all_binned_labs_vitals_test.csv'), index=False)

2319725


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

2732119
2184675
2732119


In [83]:
df_new.pat_enc_csn_id_coded.nunique()

51845

### View

In [82]:
for feature in look_up_table_df['features'].unique():
    print(look_up_table_df[look_up_table_df['features'] == feature])

   features bins values     
                    min  max
0       ALB    0    0.9  2.5
1       ALB    1    2.6  2.9
2       ALB    2    3.0  3.1
3       ALB    3    3.2  3.3
4       ALB    4    3.4  3.5
5       ALB    5    3.6  3.7
6       ALB    6    3.8  3.9
7       ALB    7    4.0  4.1
8       ALB    8    4.2  4.3
9       ALB    9    4.4  6.2
10      ALB   10    6.4  6.4
   features bins values        
                    min     max
11      ALK    0   14.0    56.0
12      ALK    1   57.0    65.0
13      ALK    2   66.0    73.0
14      ALK    3   74.0    82.0
15      ALK    4   83.0    91.0
16      ALK    5   92.0   102.0
17      ALK    6  103.0   118.0
18      ALK    7  119.0   145.0
19      ALK    8  146.0   215.0
20      ALK    9  216.0  3500.0
   features bins values        
                    min     max
21      ALT    0    5.0    12.0
22      ALT    1   13.0    16.0
23      ALT    2   17.0    19.0
24      ALT    3   20.0    23.0
25      ALT    4   24.0    27.0
26      ALT    